<h1><font color="#113D68" size=5>X. Deep Learning para Procesamiento del Lenguaje Natural</font></h1>



<h1><font color="#113D68" size=6>3. Proyecto - Desarrollar un Modelo CNN n-gramas para análisis de reseñas</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Conjunto de datos de reseñas de películas](#section1)
* [2. Preparación de datos](#section2)
    * [2.1. Dividir en conjuntos de train y test](#section21)
    * [2.2. Revisiones de carga y limpieza](#section22)
    * [2.3. Limpiar todas las reseñas y guardar](#section23)
* [3. Desarrollar modelo multicanal](#section3)
    * [3.1. Codificar datos](#section31)
    * [3.2. Definir el modelo](#section32)
    * [3.3. Mostrar los resultados](#section33)
* [4. Evaluar modelo](#section4)
* [5. Extensiones](#section5)


---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

Un modelo de aprendizaje profundo estándar para la clasificación de texto y el análisis de sentimientos utiliza una capa de incrustación de palabras y una red neuronal convolucional unidimensional. El modelo se puede expandir usando múltiples redes neuronales convolucionales paralelas que leen el documento fuente usando diferentes tamaños de kernel. Esto, en efecto, crea una red neuronal convolucional multicanal para texto que lee texto con diferentes tamaños de n-gramas (grupos de palabras). 

En este tutorial, descubrirá cómo desarrollar una red neuronal convolucional multicanal para la predicción de sentimientos en los datos de revisión de películas de texto. Después de completar este tutorial, sabrás:
- Cómo preparar datos de texto de reseñas de películas para el modelado.
- Cómo desarrollar una red neuronal convolucional multicanal para texto en Keras.
- Cómo evaluar un modelo de ajuste en datos no etiquetados de reseñas de películas.

Este tutorial se divide en las siguientes partes:
1. Conjunto de datos de revisión de películas.
2. Preparación de datos.
3. Desarrollar modelo multicanal.
4. Evaluar modelo.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Conjunto de datos de reseñas de películas</font>

En este tutorial, utilizaremos el conjunto de datos de revisión de películas. Este conjunto de datos diseñado para el análisis de sentimientos se describió anteriormente en el Capítulo 9. 

Después de descomprimir el archivo, tendrá un directorio llamado `txt_sentoken` con dos subdirectorios que contienen el texto `neg` y `pos` para reseñas negativas y positivas. Las revisiones se almacenan una por archivo con una convención de nomenclatura `cv000` a `cv999` para cada `neg` y `pos`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Desacargar el dataset [Review polarity](https://raw.githubusercontent.com/jbrownlee/Datasets/master/review_polarity.tar.gz)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Preparación de datos</font>

Nota: La preparación del conjunto de datos de reseñas de películas se describió por primera vez en capítulos anteriores. En esta sección, veremos 3 cosas:
1. Separación de datos en conjuntos de train y test.
2. Cargar y limpiar los datos para eliminar puntuación y números.
3. Limpiar todas las reseñas y guardarlas.

<a id="section21"></a>
# <font color="#004D7F" size=5>2.1. Dividir en conjuntos de train y test</font>

Pretendemos que estamos desarrollando un sistema que puede predecir el sentimiento de una reseña textual de una película como positivo o negativo. Esto significa que después de que se desarrolle el modelo, necesitaremos hacer predicciones sobre nuevas revisiones textuales. Esto requerirá que se realice la misma preparación de datos en esas revisiones nuevas que se realiza en los datos de entrenamiento para el modelo. Nos aseguraremos de que esta restricción se incorpore en la evaluación de nuestros modelos al dividir los conjuntos de datos de entrenamiento y prueba antes de cualquier preparación de datos. Esto significa que cualquier conocimiento sobre los datos en el conjunto de prueba que podría ayudarnos a preparar mejor los datos (por ejemplo, las palabras utilizadas) no está disponible en la preparación de los datos utilizados para entrenar el modelo.

Dicho esto, utilizaremos las últimas 100 reseñas positivas y las últimas 100 reseñas negativas como conjunto de prueba (100 reseñas) y las 1800 reseñas restantes como conjunto de datos de entrenamiento. Este es un tren del 90%, división del 10% de los datos. La división se puede imponer fácilmente mediante el uso de los nombres de archivo de las revisiones donde las revisiones denominadas 000 a 899 son para datos de entrenamiento y las revisiones denominadas 900 en adelante son para prueba

<a id="section22"></a>
# <font color="#004D7F" size=5>2.2. Revisiones de carga y limpieza</font>

Los datos de texto ya están bastante limpios; no se requiere mucha preparación. Sin atascarnos demasiado en los detalles, prepararemos los datos de la siguiente manera:
- Fichas divididas en espacios en blanco.
- Eliminar toda la puntuación de las palabras.
- Eliminar todas las palabras que no estén compuestas únicamente por caracteres alfabéticos.
- Eliminar todas las palabras que son palabras vacías conocidas.
- Eliminar todas las palabras que tengan una longitud $\leq$ 1 carácter.

Podemos poner todos estos pasos en una función llamada `clean_doc()` que toma como argumento el texto sin formato cargado desde un archivo y devuelve una lista de tokens limpios. También podemos definir una función `load_doc()` que cargue un documento desde un archivo listo para usar con la función `clean_doc()`. A continuación se muestra un ejemplo de limpieza de la primera crítica positiva.

In [9]:
from nltk.corpus import stopwords
import string
import re

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load the document
filename = 'data/txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

Ejecutar el ejemplo imprime una larga lista de tokens limpios. Hay muchos más pasos de limpieza que queramos explorar y los dejo como ejercicios adicionales.

<a id="section23"></a>
# <font color="#004D7F" size=5>2.3. Limpiar todas las reseñas y guardar</font>

Ahora podemos usar la función para limpiar reseñas y aplicarla a todas las reseñas. Para hacer esto, desarrollaremos una nueva función llamada `process_docs()` a continuación que recorrerá todas las revisiones en un directorio, las limpiará y las devolverá como una lista. También agregaremos un argumento a la función para indicar si la función está procesando revisiones de entrenamiento o prueba, de esa manera los nombres de archivo se pueden filtrar (como se describió anteriormente) y solo se limpiarán y devolverán las revisiones de entrenamiento o prueba solicitadas.

In [10]:
# load all docs in a directory
def process_docs(directory, is_train):
	documents = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip any reviews in the test set
		if is_train and filename.startswith('cv9'):
			continue
		if not is_train and not filename.startswith('cv9'):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load the doc
		doc = load_doc(path)
		# clean doc
		tokens = clean_doc(doc)
		# add to list
		documents.append(tokens)
	return documents

Podemos llamar a esta función con revisiones de entrenamiento negativas. También necesitamos etiquetas para el train y test. Sabemos que tenemos 900 documentos de capacitación y 100 documentos de prueba. Podemos usar una lista de comprensión de Python para crear las etiquetas para las revisiones negativas _(0)_ y positivas _(1)_ para los conjuntos de entrenamiento y prueba. La siguiente función denominada `load_clean_dataset()` cargará y limpiará el texto de la reseña de la película y también creará las etiquetas para las reseñas.

In [11]:
# load and clean a dataset
def load_clean_dataset(is_train):
	# load documents
	neg = process_docs('data/txt_sentoken/neg', is_train)
	pos = process_docs('data/txt_sentoken/pos', is_train)
	docs = neg + pos
	# prepare labels
	labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
	return docs, labels

Finalmente, queremos guardar los conjuntos train y test preparados en un archivo para que podamos cargarlos más tarde para el modelado y la evaluación del modelo. La función debajo llamada `save_dataset()` guardará un conjunto de datos preparado dado (elementos `X` e `y`) en un archivo usando la API `pickle` (esta es la API estándar para guardar objetos en Python).

In [12]:
# save a dataset to file
def save_dataset(dataset, filename):
	dump(dataset, open(filename, 'wb'))
	print('Saved: %s' % filename)

Ahora podemos llamar a las funciones para ejecutar nuestro ejemplo y que queden los corpus limpios.

In [13]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from pickle import dump

# load and clean all reviews
train_docs, ytrain = load_clean_dataset(True)
test_docs, ytest = load_clean_dataset(False)
# save training datasets
save_dataset([train_docs, ytrain], 'train.pkl')
save_dataset([test_docs, ytest], 'test.pkl')

Saved: train.pkl
Saved: test.pkl


Al ejecutar el ejemplo, se limpian los documentos de revisión de películas de texto, se crean etiquetas y se guardan los datos preparados para los conjuntos de datos de entrenamiento y prueba en train.pkl y test.pkl respectivamente. Ahora estamos listos para desarrollar nuestro modelo.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Desarrollar modelo multicanal</font>

En esta sección, desarrollaremos una red neuronal convolucional multicanal para el problema de predicción del análisis de reseñas. Esta sección se divide en 3 partes:
1. Codificar datos.
2. Definir modelo.
3. Mostrar los resultados.

<a id="section31"></a>
# <font color="#004D7F" size=5>3.1. Codificar datos</font>

El primer paso es cargar el conjunto de datos de entrenamiento limpio. Se puede llamar a la función denominada `load_dataset()` a continuación para cargar el conjunto de datos de entrenamiento en pickle.

In [6]:
# load a clean dataset
def load_dataset(filename):
	return load(open(filename, 'rb'))

A continuación, debemos colocar un `Tokenizador` de Keras en el conjunto de datos de entrenamiento. Usaremos este tokenizador para definir el vocabulario de la capa `Embedding` y codificar los documentos de revisión como números enteros. La función `create_tokenizer()` a continuación creará un `Tokenizer` dada una lista de documentos.

In [7]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

También necesitamos saber la longitud máxima de las secuencias de entrada como entrada para el modelo y rellenar todas las secuencias a la longitud fija. La función `max_length()` calculará la longitud máxima (número de palabras) para todas las revisiones en el conjunto de datos de entrenamiento.

In [8]:
# calculate the maximum document length
def max_length(lines):
    #return max([len(s.split()) for s in lines])
    return max([len(s) for s in lines])

También necesitamos saber el tamaño del vocabulario para la capa `Embedding`. Esto se puede calcular a partir del `Tokenizer` preparado, de la siguiente manera:
```python
    # calculate vocabulary size
    vocab_size = len(tokenizer.word_index) + 1
```

Finalmente, podemos codificar enteros y rellenar el texto limpio de la reseña de la película. La función de abajo
`encode_text()` codificará y rellenará los datos de texto a la longitud máxima de revisión.

In [9]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

<a id="section32"></a>
# <font color="#004D7F" size=5>3.2. Definir el modelo</font>

Un modelo estándar para la clasificación de documentos es usar una capa `Embedding` como entrada, seguida de una red neuronal convolucional unidimensional, una capa de agrupación y luego una capa de salida de predicción. El tamaño del kernel en la capa convolucional define el número de palabras a considerar como la convolución se pasa a través del documento de texto de entrada, proporcionando un parámetro de agrupación. Una red neuronal convolucional multicanal para la clasificación de documentos implica el uso de múltiples versiones del modelo estándar con kernels de diferentes tamaños. Esto permite que el documento se procese a diferentes resoluciones o diferentes n-gramas (grupos de palabras) a la vez, mientras que el modelo aprende cómo integrar mejor estas interpretaciones.

Este enfoque fue descrito por primera vez por Yoon Kim en su artículo de 2014 titulado _Convolutional Neural Networks for Sentence Classification_. En el documento, Kim experimentó con capas incrustadas estáticas y dinámicas (actualizadas), podemos simplificar el enfoque y, en su lugar, centrarnos solo en el uso de diferentes tamaños de kernel. Este enfoque se comprende mejor con un diagrama tomado del artículo de Kim, consulte el Capítulo
anteriores.

En Keras, se puede definir un modelo de entrada múltiple utilizando la API funcional. Definiremos un modelo con tres canales de entrada para procesar 4-gramas, 6-gramas y 8-gramas de texto de reseñas de películas. Cada canal se compone de los siguientes elementos:
- Capa `Input` que define la longitud de las secuencias de entrada.
- Capa `Embedding` ajustada al tamaño del vocabulario y representaciones de valor real de 100 dimensiones.
- Capa `Conv1D` con 32 filtros y un tamaño de kernel establecido en la cantidad de palabras para leer a la vez.
- Capa `MaxPooling1D` para consolidar la salida de la capa convolucional.
- Capa `Flatten` para reducir la salida tridimensional a bidimensional para la concatenación.
La salida de los tres canales se concatena en un solo vector y se procesa mediante una capa `Dense` y una capa de
salida. La siguiente función define y devuelve el modelo. Como parte de la definición del modelo, se imprime un resumen del modelo definido y se crea un gráfico del modelo y se guarda en un archivo.

In [18]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(32, 4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D()(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(32, 6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D()(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(32, 8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D()(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	model.summary()
	plot_model(model, show_shapes=True, to_file='model.png')
	return model

A continuación, se guarda un gráfico del modelo definido en un archivo con el nombre `model.png`.
  <img src="img/model_53.png" width="500" height="500" alt="CNN NLP">

<a id="section33"></a>
# <font color="#004D7F" size=5>3.3. Mostrar los resultados</font>

Ahora podemos ver el impacto de nuestra red neurnoal

In [21]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate

# load training dataset
trainLines, trainLabels = load_dataset('train.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length    
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(trainLabels), epochs=7, batch_size=16)
# save the model
model.save('model.h5')

Max document length: 1380
Vocabulary size: 44277


2022-09-13 10:57:48.555812: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-13 10:57:48.555964: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (manwest-PC): /proc/driver/nvidia/version does not exist
2022-09-13 10:57:48.557748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1380)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1380, 100)    4427700     ['input_1[0][0]']                
                                                                                              

Ejecutar el ejemplo primero imprime un resumen del conjunto de datos de entrenamiento preparado.

El modelo se ajusta relativamente rápido y parece mostrar una buena habilidad en el conjunto de datos de entrenamiento.

Se guarda un gráfico del modelo definido en un archivo, que muestra claramente los tres canales de entrada para el modelo.

El modelo se ajusta a varias épocas y se guarda en el archivo `model.h5` para una evaluación posterior.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar modelo</font>

En esta sección, podemos evaluar el modelo de ajuste al predecir la reseña en todas las revisiones en el conjunto de datos de prueba no etiquetados. Usando las funciones de carga de datos desarrolladas en la sección anterior, podemos cargar y codificar los conjuntos de datos de entrenamiento y prueba.

In [13]:
from pickle import load
from keras.models import load_model
from numpy import array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# load datasets
trainLines, trainLabels = load_dataset('train.pkl')
testLines, testLabels = load_dataset('test.pkl')
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
print('Max document length: %d' % length)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
testX = encode_text(tokenizer, testLines, length)
# load the model
model = load_model('model.h5')

Max document length: 1380
Vocabulary size: 44277


2022-09-13 12:21:51.113448: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-13 12:21:51.113573: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (manwest-PC): /proc/driver/nvidia/version does not exist
2022-09-13 12:21:51.114868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Una vez cargado el modelo guardado podemos evaluarlo tanto en los conjuntos de datos de entrenamiento como de prueba. 

In [14]:
# evaluate model on training dataset
_, acc = model.evaluate([trainX,trainX,trainX], array(trainLabels), verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset dataset
_, acc = model.evaluate([testX,testX,testX], array(testLabels), verbose=0)
print('Test Accuracy: %.2f' % (acc*100))

Train Accuracy: 100.00
Test Accuracy: 87.50


Ejecutar el ejemplo imprime la habilidad del modelo en los conjuntos de datos de entrenamiento y prueba. Podemos ver que, como era de esperar, la habilidad en el conjunto de datos de entrenamiento es excelente, aquí con una precisión del 100%.

También podemos ver que la habilidad del modelo en el conjunto de datos de prueba no etiquetados también es muy impresionante, logrando un 88,5%, que está por encima de la habilidad del modelo informado en el documento de 2014.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Extensiones</font>

En esta sección se enumeran algunas ideas para ampliar el tutorial que tal vez desee explorar.
- __Diferentes n-gramas__. Explore el modelo cambiando el tamaño del kernel (número de n-gramas) que utilizan los canales en el modelo para ver cómo afecta la habilidad del modelo.
- __Más o menos canales__. Explore el uso de más o menos canales en el modelo y vea cómo afecta la habilidad del modelo.
- __Incrustación compartida__. Explore configuraciones donde cada canal comparte la misma palabra incrustada e informe sobre el impacto en la habilidad del modelo.
- __Red más profunda__. Las redes neuronales convolucionales funcionan mejor en visión artificial cuando son más profundas. Explore el uso de modelos más profundos aquí y vea cómo afecta la habilidad del modelo.
- __Secuencias truncadas__. Rellenar todas las secuencias con la duración de la secuencia más larga puede ser extremo si la secuencia más larga es muy diferente a todas las demás revisiones. Estudie la distribución de la duración de las reseñas y trunque las reseñas a una duración media.
- __Vocabulario truncado__. Eliminamos las palabras que aparecían con poca frecuencia, pero aún teníamos un gran vocabulario de más de 25,000 palabras. Explore aún más la reducción del tamaño del vocabulario y el efecto en la habilidad del modelo.
- __Épocas y tamaño de lote__. El modelo parece ajustarse rápidamente al conjunto de datos de entrenamiento. Explore configuraciones alternativas de la cantidad de épocas de entrenamiento y el tamaño del lote y use el conjunto de datos de prueba como un conjunto de validación para elegir un mejor punto de parada para entrenar el modelo.
- __Pre-entrenamiento e incrustación__. Explore el entrenamiento previo de una palabra Word2Vec incrustada en el modelo y el impacto en la habilidad del modelo con y sin ajustes adicionales durante el entrenamiento.
- __Utilice la incrustación de guantes__. Explore la carga de la incrustación de GloVe previamente entrenada y el impacto en la habilidad del modelo con y sin más ajustes durante el entrenamiento.
- __Entrenamiento del Modelo Final__. Entrene un modelo final con todos los datos disponibles y utilícelo para hacer predicciones sobre reseñas de películas ad hoc reales de Internet.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>